In [1]:
# Import necessary libraries
import os
import json
from pathlib import Path
import numpy as np
import pandas as pd
import geopandas as gpd
import plotly.graph_objects as go
import dash
from dash import Dash, html, dcc
from dash.dependencies import Input, Output, State
import dash_mantine_components as dmc
import dash_bootstrap_components as dbc
import dash_daq as daq
from flask import Flask
import datetime as dt
from shapely.geometry import MultiPoint, mapping, Polygon, MultiPolygon, LineString, MultiLineString
import folium
import hdbscan
from scipy.spatial import ConvexHull


In [2]:
fn = r'C:\Users\markd\projects\Hamilton County Homes\kx-ohio-school-districts-SHP'
os.chdir(fn)
# open it...
geodf = gpd.read_file(list(Path.cwd().glob("ohio*.shp"))[0])


In [3]:
fn = r'C:\Users\markd\projects\Hamilton County Homes'
os.chdir(fn)
# open it...
homes = gpd.read_file(list(Path.cwd().glob("finalsold2009_2023.csv"))[0])


In [4]:
# Convert market land value from string to float, cleaning up currency formatting
homes['market_land_value_num'] = homes['market_land_value'].replace('[\$,]', '', regex=True).astype(float)

# Group the data by 'latitude' and calculate the average market land value for each latitude
average_land_value_by_latitude = homes.groupby('latitude')['market_land_value_num'].mean().reset_index()

# Find the latitudes with the highestdata average market land value
highest_land_values = average_land_value_by_latitude.sort_values(by='market_land_value_num', ascending=False).head()

highest_land_values

,latitude,market_land_value_num
89,39.1465623,242610.0
27,39.1436349,239560.0
185,39.186685,221980.0
652,39.2495471,214620.0
91,39.1466163,209820.0


In [5]:
homes = homes.drop(columns=['foreclosure','cauv_value','tif_value','exempt_value','new_address','missing_house_number','num_amount'])

In [6]:
homes['month_yr']= pd.to_datetime(homes['transfer_date']).dt.strftime('%m/%Y')

In [7]:
convert_dict = {
    'year_built':int,
    'total_rooms':int,
    'bedrooms':int,
    'full_baths':int,
    'half_baths':int,
    'num_parcels_sold':int,
    'acreage':float,
    'effective_tax_rate':float,
    'market_land_value':int,
    'market_improvement_value':int,
    'market_total_value':int,
    'abated_value':int}
homes = homes.astype(convert_dict)
homes.transfer_date = pd.to_datetime(homes.transfer_date)
homes.amount = homes.amount.replace('[\$,]', '', regex=True).astype('int')
homes.taxes_paid = homes.taxes_paid.replace('[\$,]', '', regex=True).astype('float')
homes.annual_tax = homes.annual_tax.replace('[\$,]', '', regex=True).astype('float')


In [8]:
homes['age_of_house'] = dt.datetime.today().year - homes.year_built
homes['city'] = homes['formatted_address'].str.extract(r'(\b[a-zA-Z]+(?:\s+[a-zA-Z]+)*),\sOH\b', expand=False)
homes.finsqft = homes.finsqft.astype('int64')
homes['latitude'] = homes['latitude'].astype('float')
homes['longitude'] = homes['longitude'].astype('float')

In [9]:
geodf = geodf.rename(columns={'ID':'district_id'})
geodf['created_da'] = pd.to_datetime(geodf['created_da'])
geodf['last_edi_1'] = pd.to_datetime(geodf['last_edi_1'])
geodf = geodf.drop(columns=['created_da','last_edi_1'])

In [10]:
district_idn_map = {
    'SYCAMORE CSD': '044867',
    'WYOMING CSD': '045146',
    'MADEIRA CSD': '044289',
    'MARIEMONT CSD': '044313',
    'LOVELAND CSD':'044271'
    # Add more districts and colors as needed
}

homes['idn'] = homes.school_district.map(district_idn_map)

In [11]:
home_type_map = {
    '510': 'Single Family',
    '550': 'Condo',
    '555': 'Townhome',
    '520': 'Two Family',
}

homes['home_type'] = homes.use.map(home_type_map)

In [13]:
homes

,field_1,parcel_number,address,finsqft,use,year_built,transfer_date,amount,total_rooms,bedrooms,...,longitude,latitude,year,month,city,market_land_value_num,month_yr,age_of_house,idn,home_type
0,0,521-0009-0066-00,101 Fieldstone Dr,1426,510,1954,2009-06-17,120000,6,3,...,-84.315106,39.164736,2009,6,Terrace Park,101300.0,06/2009,70,044313,Single Family
1,1,523-0006-0037-00,3914 Germania Ave,1066,510,1930,2009-06-11,131750,6,3,...,-84.393614,39.146012,2009,6,Cincinnati,57530.0,06/2009,94,044313,Single Family
2,2,523-0006-0103-00,3714 Lonsdale St,1144,510,1955,2009-06-11,148500,5,3,...,-84.390158,39.143411,2009,6,Cincinnati,68310.0,06/2009,69,044313,Single Family
3,3,523-0006-0222-00,6110 Elder St,1436,510,1930,2009-06-24,120000,6,3,...,-84.390794,39.146337,2009,6,Cincinnati,56400.0,06/2009,94,044313,Single Family
4,4,523-0006-0304-00,3811 Carlton Ave,1064,510,1918,2009-05-05,105000,5,3,...,-84.388793,39.145544,2009,5,Cincinnati,57530.0,05/2009,106,044313,Single Family
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1265,1265,621-0024-0097-00,110 104 Carrington Ln,1412,550,1992,2023-06-05,250000,6,2,...,-84.301031,39.286614,2023,6,Loveland,12000.0,06/2023,32,044867,Condo
1266,1266,621-0024-0102-00,120 101 Carrington Ln,1095,550,1992,2023-06-02,177000,5,2,...,-84.301031,39.286614,2023,6,Loveland,12000.0,06/2023,32,044867,Condo
1267,1267,621-0024-0117-00,130 104 Carrington Ln,1412,550,1992,2023-04-07,200000,6,2,...,-84.301031,39.286614,2023,4,Loveland,12000.0,04/2023,32,044867,Condo
1268,1268,621-0024-0121-00,130 208 Carrington Ln,1412,550,1992,2023-04-28,210000,6,2,...,-84.301031,39.286614,2023,4,Loveland,12000.0,04/2023,32,044867,Condo


In [86]:



# Constants (replace with your actual dataset paths)
FN = r'C:\Users\markd\projects\Hamilton County Homes\kx-ohio-school-districts-SHP'
district_names = homes.school_district.unique()
ODE_IRN_LIST = ['044867','045146','044289','044313','044271']
MAP_CENTER = dict(lat=39.2127649, lon=-84.3831728)
years = homes.year.unique()
home_types = homes.home_type.unique()
district_idn_map = {
    'SYCAMORE CSD': '044867',
    'WYOMING CSD': '045146',
    'MADEIRA CSD': '044289',
    'MARIEMONT CSD': '044313',
    'LOVELAND CSD':'044271'
    # Add more districts and colors as needed
}
annotations = []
COLORSCALE = {
     '044867':'rgba(0, 38, 66,.1)',    
     '045146':'rgba(132, 0, 50,.1)',
     '044289':'rgba(0, 187, 249,.1)',
     '044313':'rgba(0, 245, 212,.1)',
    '044271':'rgba(175, 43, 191,.1)',
}

district_color_map = {
    'SYCAMORE CSD': ' rgba(132, 0, 50,1)',
    'WYOMING CSD': 'rgba(0, 38, 66,1)',
    'MADEIRA CSD': 'rgba(0, 187, 249,1)',
    'MARIEMONT CSD': 'rgba(0, 245, 212,1)',
    'LOVELAND CSD':'rgba(175, 43, 191,1)',
    # Add more districts and colors as needed
}


df = homes
df2 = pd.pivot_table(homes,
              index = ['school_district','year'],
              values=['amount','parcel_number']
              , aggfunc={'amount':'median','parcel_number':'count'}).reset_index()

MIN = homes.amount.min()
MAX = homes.amount.max()
STEP = (homes.amount.max() - homes.amount.min())/10

year_dict = [{'label':'Select All','value':'select-all'}]+[{'label': str(year), 'value': year} for year in years]
home_type_dict = [{'label':'Select All','value':'select-all'}]+[{'label': type, 'value': type} for type in home_types]
district_dict = [{'label':'Select All','value':'select-all'}]+[{'label': district.split(' ')[0].title(), 'value': district} for district in district_names]

year_options = ['select-all']+[year for year in years]
home_type_options = ['select-all']+[type for type in home_types]
district_options = ['select-all']+[district for district in district_names]

# Initialize the app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Layout with three sections, a filter section at the top, a KPI section to the left, and a map on the right
app.layout = html.Div([ 
                #Filter Section
                html.Div([       
                    # Filter button section (Toggle Dropdown for Years)
                    html.Div(
                        dbc.DropdownMenu(
                            label="Years",
                            children=[
                                dcc.Checklist(
                                    id='year-checklist',
                                    options=year_dict,
                                    value=year_options,
                                    inputStyle={'margin-left': '5px',
                                                'margin-right': '5px',
                                                'margin-top': '5px',
                                                'margin-bottom': '5px',
                                                },  # Optional styling for spacing
                                    style={'padding': '10px'}  # Add padding inside the dropdown
                                )
                            ],
                            direction="down",
                            id="year-dropdown-menu",
                            className="custom-dropdown",
                            right=False,  # Align dropdown to the left
                            style={
                                'padding': '10px',
                                'display': 'inline-block'  # Ensure dropdown takes only necessary width
                            }
                        ),
                        style={'display': 'inline-block','padding-left':'5px'}  # Add a small gap between buttons
                    ),
                    html.Div(
                        dbc.DropdownMenu(
                            label="District",
                            children=[
                                dcc.Checklist(
                                    id='district-checklist',
                                    options=district_dict,
                                    value=district_options,
                                    inputStyle={'margin-left': '5px',
                                                'margin-right': '5px',
                                                'margin-top': '5px',
                                                'margin-bottom': '5px',
                                                },  # Optional styling for spacing
                                    style={'padding': '10px'}  # Add padding inside the dropdown
                                )
                            ],
                            direction="down",
                            id="district-dropdown-menu",
                            className="custom-dropdown",
                            right=False,  # Align dropdown to the left
                            style={
                                'padding': '10px',
                                'display': 'inline-block'  # Ensure dropdown takes only necessary width
                            }
                        ),
                        style={'display': 'inline-block','padding-left':'5px'}  # Ensure elements stay in the same line
                    ),
                    html.Div(
                        dbc.DropdownMenu(
                            label="Home Type",
                            children=[
                                dcc.Checklist(
                                    id='home-type-checklist',
                                    options=home_type_dict,
                                    value=home_type_options,
                                    inputStyle={'margin-left': '5px',
                                                'margin-right': '5px',
                                                'margin-top': '5px',
                                                'margin-bottom': '5px',
                                                },  # Optional styling for spacing
                                    style={'padding': '10px'}  # Add padding inside the dropdown
                                )
                            ],
                            direction="down",
                            id="home-type-dropdown-menu",
                            className="custom-dropdown",
                            right=False,  # Align dropdown to the left
                            style={
                                'padding': '10px',
                                'display': 'inline-block'  # Ensure dropdown takes only necessary width
                            }
                        ),
                        style={'display': 'inline-block','padding-left':'5px'}  # Add a small gap between buttons
                    ),
                    html.Div(
                        dbc.DropdownMenu(
                            label="District Boundary",
                            children=[
                            dcc.RadioItems([{'label':'On', 'value':'On'},
                                            {'label':'Off','value':'Off'}],
                                value = 'Off',
                                inputStyle={'margin-left': '5px',
                                            'margin-right': '5px',
                                            'margin-top': '5px',
                                            'margin-bottom': '5px',
                                            },  # Optional styling for spacing
                                id='district-boundary',
                                    )                            
                            ],
                            direction="down",
                            id="district-boundary-dropdown",
                            className="custom-dropdown",
                            right=False,  # Align dropdown to the left
                            style={
                                'padding': '10px',
                                'display': 'inline-block'  # Ensure dropdown takes only necessary width
                            }
                        ),
                        style={'display': 'inline-block','padding-left':'5px'}  # Add a small gap between buttons
                    ),
                    html.Div(
                        dbc.DropdownMenu(
                            label="Show Homes",
                            children=[
                            dcc.RadioItems([{'label':'On', 'value':'On'},
                                            {'label':'Off','value':'Off'}],
                                value = 'Off',
                                inputStyle={'margin-left': '5px',
                                            'margin-right': '5px',
                                            'margin-top': '5px',
                                            'margin-bottom': '5px',
                                            },  # Optional styling for spacing
                                id='home-markers',
                                    )                            
                            ],
                            direction="down",
                            id="home-marker-dropdown",
                            className="custom-dropdown",
                            right=False,  # Align dropdown to the left
                            style={
                                'padding': '10px',
                                'display': 'inline-block'  # Ensure dropdown takes only necessary width
                            }
                        ),
                        style={'display': 'inline-block','padding-left':'5px'}  # Add a small gap between buttons
                    ),
                    html.Div(
                        dbc.DropdownMenu(
                            label="House Sales Hotspots",
                            children=[
                            dcc.RadioItems([{'label':'On', 'value':'On'},
                                            {'label':'Off','value':'Off'}],
                                value = 'Off',
                                inputStyle={'margin-left': '5px',
                                            'margin-right': '5px',
                                            'margin-top': '5px',
                                            'margin-bottom': '5px',
                                            },  # Optional styling for spacing
                                id='cluster-boundary',
                                    )                            
                            ],
                            direction="down",
                            id="cluster-boundary-dropdown",
                            className="custom-dropdown",
                            right=False,  # Align dropdown to the left
                            style={
                                'padding': '10px',
                                'display': 'inline-block'  # Ensure dropdown takes only necessary width
                            }
                        ),
                        style={'display': 'inline-block','padding-left':'5px'}  # Add a small gap between buttons
                    ),
                    html.Div([
                        html.Label("Price Range", style={'color': '#3a3838', 'text-align': 'left','padding-left':'20px'}),
                        html.Div(
                            dcc.RangeSlider(
                                MIN, MAX, STEP,
                                count=STEP,
                                value=[MIN, MAX],
                                id='sale-price-slider',
                            ), 
                            style={'flex-grow': '1'}),  # Apply flex-grow to the container div, not the slider itself
                        ], style={'display': 'inline-block', 'width': '100%','color': '#3a3838','font-weight':'bold','text-align':'left','padding-left':'5px'})
                    ], style={'display': 'flex', 'text-align':'left','padding-top':'20px'}),  # Use Flexbox to align elements in a row
                
                html.Div([
                    # Left Section: KPIs, Line Graph, and Bar Graph
                    html.Div([
                        # KPI Section
                        html.Div([
                            dcc.Graph(id='count-kpi', style={'width': '50%'}),  # KPI indicator 1
                            dcc.Graph(id='kpi', style={'width': '50%'})  # KPI indicator 2
                        ], style={'display': 'flex', 'flex-direction': 'row', 'color': '#3a3838','height':'30%'}),

                        # Line Graph Section (below KPIs)
                        html.Div([
                            dcc.Graph(id='yoy-change-graph', style={'width': '80%'})  # Line graph
                        ], style={'display': 'flex', 'flex-direction': 'column', 'padding-top': '0px','margin-top':'0px','height':'35%'}),  # Full width for the line graph
                        
                        # Bar Graph Section (below Line Graph)
                        html.Div([
                            dcc.Graph(id='home-count-graph', style={'width': '80%'})  # Line graph
                        ], style={'display': 'flex', 'flex-direction': 'column', 'padding-top': '0px','margin-top':'0px','height':'35%'}),  # Full width for the line graph
                    ], style={ 'width': '30%','padding-right': '0px','margin-top':'0px'}),

                    # Right Section: Map
                    html.Div([
                        dcc.Graph(id='map', style={'width': '100%'})  # Map takes up the entire right section
                    ], style={'display': 'flex', 'width': '70%','height':'60%'})
                ], style={'display': 'flex', 'width': '100%'})
            ], style={'position': 'relative', 'width': '100%'})  # Ensure map takes full width

@app.callback(
    Output('home-type-checklist', 'value'),
    [Input('home-type-checklist', 'value'),],
    prevent_initial_call=True
)
def update_home_type_checklist(selected_values):
    if 'select-all' in selected_values:
        # If 'all' is selected, return all options (including 'Select All')
        if 'select-all' in selected_values:
            return home_type_options
        else:
            # If "select-all" is selected along with other items, deselect everything
            return []
    elif ('select-all' not in selected_values) and (len(selected_values)==len(home_type_options)-1):
        return []
    else:
        # If "select-all" is unchecked, only return other selected items
        return [x for x in selected_values if x != 'select-all']
        

@app.callback(
    Output('year-checklist', 'value'),
    [Input('year-checklist', 'value'),],
    prevent_initial_call=True
)
def update_year_checklist(selected_values):
    if 'select-all' in selected_values:
        # If 'all' is selected, return all options (including 'Select All')
        if 'select-all' in selected_values:
            return year_options
        else:
            # If "select-all" is selected along with other items, deselect everything
            return []
    elif ('select-all' not in selected_values) and (len(selected_values)==len(year_options)-1):
        return []
    else:
        # If "select-all" is unchecked, only return other selected items
        return [x for x in selected_values if x != 'select-all']
        

@app.callback(
    Output('district-checklist', 'value'),
    [Input('district-checklist', 'value'),],
    prevent_initial_call=True
)
def update_district_checklist(selected_values):
    if 'select-all' in selected_values:
        # If 'all' is selected, return all options (including 'Select All')
        if 'select-all' in selected_values:
            return district_options
        else:
            # If "select-all" is selected along with other items, deselect everything
            return []
    elif ('select-all' not in selected_values) and (len(selected_values)==len(district_options)-1):
        return []
    else:
        # If "select-all" is unchecked, only return other selected items
        return [x for x in selected_values if x != 'select-all']
    
@app.callback(
    Output('count-kpi', 'figure'),
    [Input('sale-price-slider', 'value'),
     Input('district-checklist', 'value'),
     Input('home-type-checklist', 'value'),
     Input('year-checklist', 'value')]
)
def update_kpi(sale_price_range, district_value, home_type_value, year_value):
    # Start with the base filter for sale price range
    filtered_df = df[(df['amount'] >= sale_price_range[0]) & (df['amount'] <= sale_price_range[1])]

    # Add filters progressively only if the corresponding values are not None
    if district_value is not None:
        filtered_df = filtered_df[filtered_df['school_district'].isin(district_value)]

    if home_type_value is not None:
        filtered_df = filtered_df[filtered_df['home_type'].isin(home_type_value)]

    if year_value is not None:
        filtered_df = filtered_df[filtered_df['year'].isin(year_value)]    

    median_price = len(filtered_df['amount'])

    # Create the KPI indicator
    kpi_figure = go.Figure(go.Indicator(
        mode="number",
        value=median_price,
        number = {'font':{'size':48}},
    ))

    # Customize the layout of the KPI
    kpi_figure.update_layout(
        title = {'text': "No. Home Sales", 'font': {'size': 14}, 'x':0.5, 'y': 0.25, 'yanchor': 'bottom'},
        margin={'t': 0, 'b': 0, 'l': 0, 'r': 0},  # Reduce margins
        paper_bgcolor="rgba(0,0,0,0)",  # Transparent background
        font={'color':"#3a3838",'size':10},  # Adjust font size and color
    )

    return kpi_figure

@app.callback(
    Output('kpi', 'figure'),
    [Input('sale-price-slider', 'value'),
     Input('district-checklist', 'value'),
     Input('home-type-checklist', 'value'),
     Input('year-checklist', 'value')]
)
def update_kpi(sale_price_range, district_value,home_type_value, year_value):
    # Start with the base filter for sale price range
    filtered_df = df[(df['amount'] >= sale_price_range[0]) & (df['amount'] <= sale_price_range[1])]

    # Add filters progressively only if the corresponding values are not None
    if district_value is not None:
        filtered_df = filtered_df[filtered_df['school_district'].isin(district_value)]

    if home_type_value is not None:
        filtered_df = filtered_df[filtered_df['home_type'].isin(home_type_value)]

    if year_value is not None:
        filtered_df = filtered_df[filtered_df['year'].isin(year_value)]    
    
    # Example KPI: Total number of sales
    median_price = filtered_df['amount'].median()

    # Create the KPI indicator
    kpi_figure = go.Figure(go.Indicator(
        mode="number",  # or "number" based on your needs
        value=median_price,
        number={'font': {'size': 48}},  # Adjust the number size
        # delta={'reference': previous_value, 'position': "top"}
    ))

    # Update layout to reduce margins
    kpi_figure.update_layout(
        title = {'text': "Median Home Price", 'font': {'size': 14}, 'x':0.5, 'y': 0.25, 'yanchor': 'bottom'},
        margin={'t': 0, 'b': 0, 'l': 0, 'r': 0},  # Reduce margins
        paper_bgcolor="rgba(0,0,0,0)",  # Transparent background
        font={'color':"#3a3838",'size':10},  # Adjust font size and color
    )

    return kpi_figure

@app.callback(
    Output('yoy-change-graph', 'figure'),
    [Input('sale-price-slider', 'value'),
     Input('district-checklist', 'value'),
     Input('home-type-checklist', 'value')]
)
def update_yoy_change_lines(sale_price_range, district_value, home_type_value):

    # Handle 'select-all' option for district_value, home_type_value, and year_value
    if district_value == 'select-all' or len(district_value)==0:
        district_value = df['school_district'].unique().tolist()  # Select all districts if 'select-all'
    
    if home_type_value == 'select-all' or len(home_type_value)==0:
        home_type_value = df['home_type'].unique().tolist()  # Select all home types if 'select-all'
    
    # Ensure 'select-all' is not in district_value if it's a list
    if 'select-all' in district_value:
        district_value.remove('select-all')

    # Start with the base filter for sale price range
    filtered_df = df[(df['amount'] >= sale_price_range[0]) & (df['amount'] <= sale_price_range[1])]

    # Apply filters for district, home type, and year
    filtered_df = filtered_df[filtered_df['school_district'].isin(district_value)]
    filtered_df = filtered_df[filtered_df['home_type'].isin(home_type_value)]
    
    # Pivot table and calculate year-over-year change
    homes_pivot_df = filtered_df.pivot_table(index=['year'], values='amount', aggfunc='mean').reset_index()
    homes_pivot_df['yoy_change'] = homes_pivot_df.amount.pct_change()
    homes_pivot_df['year'] = homes_pivot_df['year'].astype('int')

    annotations = []  # Ensure annotations is initialized before appending

    # Loop through each school district and add traces and annotations
    # Get data for the specific school district
    x_data = homes_pivot_df['year'].to_list()
    y_data = homes_pivot_df['yoy_change'].to_list()

    # Custom hovertext for each point
    hover_text = [f"Year: {x}<br>YoY Change: {round(y*100, 2)}%" for x, y in zip(x_data, y_data)]
    
    # Add a scatter trace for each school district
    fig = go.Figure(go.Scatter(
            x=x_data, 
            y=y_data, 
            mode='lines', 
            line=dict(color='#003249', width=4),
            text=hover_text,
            hoverinfo='text',
            hovertemplate='%{text}'
        )
    )
    
    # Label the left side of the plot
    annotations.append(dict(
        xref='x', yref='y', x=x_data[1]-2, y=y_data[1]+0.03,
        xanchor='left', yanchor='bottom',
        text='{}%'.format(round(y_data[1] * 100, 2)),
        font=dict(family='Arial', size=14, color='#003249',weight='bold'),
        showarrow=False
        ))
    
    # Label the right side of the plot
    annotations.append(dict(
        xref='x', yref='y', x=x_data[-1]+0.1, y=y_data[-1],
        xanchor='left', yanchor='bottom',
        text='{}%'.format(round(y_data[-1] * 100, 2)),
        font=dict(family='Arial', size=14, color='#003249',weight='bold'),
        showarrow=False
        ))

    # Update the layout for x and y axes
    fig.update_xaxes(showline=False, showgrid=False, zeroline=False, 
                        tickfont=dict(
                            size=14,
                            family='Arial', # Font family
                            color='#003249',   # Font color
                            weight='bold'
                            ), linecolor='#003249', linewidth=1)
    fig.update_yaxes(showgrid=False, zeroline=False, showticklabels=False)

    # Adjust layout settings
    fig.update_layout(
        height=150,  # Adjust height based on the number of rows (districts)
        showlegend=False,
        plot_bgcolor='rgba(0, 0, 0, 0)',
        paper_bgcolor='rgba(0, 0, 0, 0)',
        margin=dict(t=30, l=30, b=30, r=30),
        title={
            'text': "Year-over-Year % Price Change",  # Title text
            'font': {
                'family': "Arial",        # Font family
                'size': 14,               # Font size
                'color': "Gray",          # Title color
                'weight': 'bold'          # Make title bold
            },
        'x': 0.5,  # Center the title
        'xanchor': 'center'
        },
        annotations=annotations
        )
    return fig

@app.callback(
    Output('home-count-graph', 'figure'),
    [Input('sale-price-slider', 'value'),
     Input('district-checklist', 'value'),
     Input('home-type-checklist', 'value')]
)
def update_home_count_graph(sale_price_range, district_value, home_type_value):

    # Handle 'select-all' option for district_value, home_type_value, and year_value
    if district_value == 'select-all' or len(district_value)==0:
        district_value = df['school_district'].unique().tolist()  # Select all districts if 'select-all'
    
    if home_type_value == 'select-all' or len(home_type_value)==0:
        home_type_value = df['home_type'].unique().tolist()  # Select all home types if 'select-all'
    
    # Ensure 'select-all' is not in district_value if it's a list
    if 'select-all' in district_value:
        district_value.remove('select-all')

    # Start with the base filter for sale price range
    filtered_df = df[(df['amount'] >= sale_price_range[0]) & (df['amount'] <= sale_price_range[1])]

    # Apply filters for district, home type, and year
    filtered_df = filtered_df[filtered_df['school_district'].isin(district_value)]
    filtered_df = filtered_df[filtered_df['home_type'].isin(home_type_value)]
    
    # Pivot table and calculate year-over-year change
    homes_pivot_df = filtered_df.pivot_table(index=['year'], values='parcel_number', aggfunc='count').reset_index()
    homes_pivot_df['year'] = homes_pivot_df['year'].astype('int')

    annotations = []  # Ensure annotations is initialized before appending

    # Loop through each school district and add traces and annotations
    # Get data for the specific school district
    x_data = homes_pivot_df['year'].to_list()
    y_data = homes_pivot_df['parcel_number'].to_list()

    y_min = min(y_data)-15
    y_max = max(y_data)+15

    # Custom hovertext for each point
    hover_text = [f"Year: {x}<br>House Sale Counts: {y}" for x, y in zip(x_data, y_data)]
    
    # Add a scatter trace for each school district
    fig = go.Figure(go.Bar(
            x=x_data, 
            y=y_data, 
            text=y_data,
            hovertext=hover_text,
            textposition='outside',
            marker={'color':'#003249'},
        )
    )
    
    # Update the layout for x and y axes
    fig.update_xaxes(showline=False, showgrid=False, zeroline=False, 
                        tickfont=dict(
                            size=14,
                            family='Arial', # Font family
                            color='#003249',   # Font color
                            weight='bold'
                            ), linecolor='#003249', linewidth=1)
    fig.update_yaxes(showgrid=False, zeroline=False, showticklabels=False)

    # Adjust layout settings
    fig.update_layout(
        height=150,  # Adjust height
        showlegend=False,
        plot_bgcolor='rgba(0, 0, 0, 0)',
        paper_bgcolor='rgba(0, 0, 0, 0)',
        margin=dict(t=50, l=30, b=0, r=30),
        title={
            'text': "House Sale Counts",  # Title text
            'font': {
                'family': "Arial",        # Font family
                'size': 14,               # Font size
                'color': "Gray",          # Title color
                'weight': 'bold'          # Make title bold
            },
        'x': 0.5,  # Center the title
        'xanchor': 'center',
        },
        yaxis = dict(
            range = [y_min,y_max]
        ),
        )
    return fig

@app.callback(
    Output('map', 'figure'),
    [Input('sale-price-slider', 'value'),
     Input('district-checklist', 'value'),
     Input('home-type-checklist', 'value'),
     Input('district-boundary', 'value'),
     Input('cluster-boundary', 'value'),
     Input('home-markers', 'value'),
     Input('year-checklist', 'value')]
)
def update_map(sale_price_range, district_value, home_type_value, boundary_value, cluster_value, show_marker_value, year_value):
    # Start with the base filter for sale price range
    filtered_df = df[(df['amount'] >= sale_price_range[0]) & (df['amount'] <= sale_price_range[1])]

    # Add filters progressively only if the corresponding values are not None
    if district_value is not None:
        filtered_df = filtered_df[filtered_df['school_district'].isin(district_value)]

    if home_type_value is not None:
        filtered_df = filtered_df[filtered_df['home_type'].isin(home_type_value)]

    if year_value is not None:
        filtered_df = filtered_df[filtered_df['year'].isin(year_value)]

    if 'On' in show_marker_value:
        # Map marker colors based on districts
        marker_colors = filtered_df['school_district'].map(district_color_map).fillna('rgba(126, 232, 250, 0.25)')
        
        # Initialize the map figure with home sales markers
        map_figure = go.Figure(go.Scattermapbox(
            lat=filtered_df['latitude'],
            lon=filtered_df['longitude'],
            text=filtered_df['school_district'],
            customdata=np.stack(
                (filtered_df['amount'], filtered_df['finsqft'], filtered_df['year'], filtered_df['address']),
                axis=-1
            ),
            mode='markers',
            marker={
                "size": 10,
                'color': marker_colors,  # Set marker colors based on the district
            },
            opacity=0.5,
            showlegend=False,  # Disable the legend
            hovertemplate='<br>'.join([
                'Amount: %{customdata[0]}',
                'Square Ft: %{customdata[1]}',
                'Year Sold: %{customdata[2]}',
                'Address: %{customdata[3]}'])
        ))

    if 'Off' in show_marker_value:
        map_figure = go.Figure(go.Scattermapbox())
    
    # If boundary checkbox is 'On', add the Choroplethmapbox trace
    if 'On' in boundary_value:
        # Create a choropleth trace
        idns = filtered_df['idn'].unique()
        geodf_filtered = geodf[geodf.ODE_IRN.isin(idns)]
        geojson_data = json.loads(geodf_filtered.to_json())
        choropleth_trace = go.Choroplethmapbox(
            geojson=geojson_data,  # GeoJSON data from geodf
            locations=geodf.index,  # Use the index of the GeoDataFrame as unique locations
            z=geodf['district_id'],  # The column with district values for coloring
            colorscale='gray',  # Use a predefined color scale
            showscale=False,  # Disable the color scale bar
            marker_opacity=0.2,  # Adjust the opacity of the fill
            marker_line_width=2,  # Width of the boundary lines
            marker_line_color='black'  # Color of the boundary lines
        )

        # Add the choropleth trace to the existing map_figure
        map_figure.add_trace(choropleth_trace)  

    # If cluster checkbox is 'On', add the cluster boundaries
    if 'On' in cluster_value:
        coords = filtered_df[['latitude', 'longitude']]
        coords_radians = np.radians(coords)

        # HDBSCAN clustering
        hdbscan_clusterer = hdbscan.HDBSCAN(min_cluster_size=5, metric='haversine')
        hdbscan_labels = hdbscan_clusterer.fit_predict(coords_radians)

        # Assign cluster labels
        filtered_df.loc[coords.index, 'cluster'] = hdbscan_labels
        clustered_data = filtered_df[filtered_df['cluster'] >= 0]

        # Iterate over clusters and create polygons (convex hulls)
        for cluster_label in clustered_data['cluster'].unique():
            cluster_points = clustered_data[clustered_data['cluster'] == cluster_label][['latitude', 'longitude']].values

            if len(cluster_points) > 2:  # Convex hull requires at least 3 points
                try:
                    hull = ConvexHull(cluster_points)
                    hull_points = cluster_points[hull.vertices]  # Get the points that form the convex hull
                    hull_coords = [(point[0], point[1]) for point in hull_points]

                    # Create scattermapbox trace for the hull
                    map_figure.add_trace(go.Scattermapbox(
                        mode="lines",
                        fill="toself",
                        lon=[point[1] for point in hull_coords],
                        lat=[point[0] for point in hull_coords],
                        line=dict(color="#1B3B6F", width=3),
                        fillcolor="rgba(27, 59, 111,0.2)",  # Fill color for the polygon
                        showlegend=False  # Disable the legend
                    ))

                except:
                    # If ConvexHull fails, fallback to plotting points
                    map_figure.add_trace(go.Scattermapbox(
                        mode="markers",
                        lon=cluster_points[:, 1],
                        lat=cluster_points[:, 0],
                        marker=dict(size=8, color="red"),
                        showlegend=False  # Disable the legend
                    ))

            else:
                # If not enough points for a hull, plot just the points
                map_figure.add_trace(go.Scattermapbox(
                    mode="markers",
                    lon=cluster_points[:, 1],
                    lat=cluster_points[:, 0],
                    marker=dict(size=8, color="red"),
                    showlegend=False  # Disable the legend
                )) 

    # Update layout for map
    map_figure.update_layout(
        mapbox_style="open-street-map",
        mapbox_center=MAP_CENTER,
        mapbox_zoom=10,
        margin={"r": 0, "t": 0, "l": 0, "b": 0},
        title=f"Map of Home Sales"
    )

    return map_figure           
  
# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


In [32]:


# Create a map centered on an approximate location (change lat/lon as needed)
map_center = [39.16, -84.39]  # Adjust to your data's center
clusters = folium.Map(location=map_center, zoom_start=12)

coords = homes[['latitude', 'longitude']]
coords_radians = np.radians(coords)

# Assuming you already have 'coords_radians' as your latitude/longitude in radians
hdbscan_clusterer = hdbscan.HDBSCAN(min_cluster_size=5, metric='haversine')

# Fit the HDBSCAN model
hdbscan_labels = hdbscan_clusterer.fit_predict(coords_radians)

# Assign HDBSCAN cluster labels back to your cleaned data
coords_radians = np.radians(coords)
homes.loc[coords_radians.index, 'cluster'] = hdbscan_labels

# Inspect the resulting clusters
print(homes[['latitude', 'longitude', 'cluster']].dropna())

# Group data by clusters
clustered_data = homes[homes['cluster'] >= 0]
idns = homes['idn'].unique()
geodf_filtered = geodf[geodf.ODE_IRN.isin(idns)]

# folium.GeoJson(geodf_filtered, name="School Districts").add_to(clusters)

# Iterate over each cluster and create a convex hull polygon
for cluster_label in clustered_data['cluster'].unique():
    cluster_points = clustered_data[clustered_data['cluster'] == cluster_label][['latitude', 'longitude']].values
    
    if len(cluster_points) > 2:  # Convex hull requires at least 3 points
        try:
            hull = ConvexHull(cluster_points)
            hull_points = cluster_points[hull.vertices]  # Get the points that form the convex hull
            
            # Convert to list of (lat, lon) tuples for folium
            hull_coords = [(point[0], point[1]) for point in hull_points]
            
            # Create a polygon and add it to the map
            folium.Polygon(hull_coords, color='green', fill=True, fill_opacity=0.4).add_to(clusters)
        
        except:
            # Fallback if ConvexHull fails (for collinear points) - just plot the points
            for point in cluster_points:
                folium.CircleMarker(location=(point[0], point[1]), radius=5, color='red').add_to(clusters)
    
    else:
        # If not enough points for a hull, plot just the points
        for point in cluster_points:
            folium.CircleMarker(location=(point[0], point[1]), radius=5, color='red').add_to(clusters)

# for idx, row in homes.iterrows():
#     # Only plot points that belong to a cluster
#     if row['cluster'] >= 0:
#         folium.CircleMarker(
#             location=(row['latitude'], row['longitude']),
#             radius=5,
#             color='green',  # Assign colors for different clusters
#             fill=True,
#             fill_opacity=0.7
#         ).add_to(clusters)

# Display the map
clusters.save('clustered_map.html')

       latitude  longitude  cluster
0     39.164736 -84.315106      0.0
1     39.146012 -84.393614      2.0
2     39.143411 -84.390158      2.0
3     39.146337 -84.390794      2.0
4     39.145544 -84.388793      2.0
...         ...        ...      ...
1265  39.286614 -84.301031     22.0
1266  39.286614 -84.301031     22.0
1267  39.286614 -84.301031     22.0
1268  39.286614 -84.301031     22.0
1269  39.285034 -84.300550     -1.0

[1270 rows x 3 columns]


In [33]:
clusters

In [35]:
homes

,field_1,parcel_number,address,finsqft,use,year_built,transfer_date,amount,total_rooms,bedrooms,...,longitude,latitude,year,month,city,market_land_value_num,month_yr,age_of_house,idn,cluster
0,0,521-0009-0066-00,101 Fieldstone Dr,1426,510,1954,2009-06-17,120000,6,3,...,-84.315106,39.164736,2009,6,Terrace Park,101300.0,06/2009,70,044313,0.0
1,1,523-0006-0037-00,3914 Germania Ave,1066,510,1930,2009-06-11,131750,6,3,...,-84.393614,39.146012,2009,6,Cincinnati,57530.0,06/2009,94,044313,2.0
2,2,523-0006-0103-00,3714 Lonsdale St,1144,510,1955,2009-06-11,148500,5,3,...,-84.390158,39.143411,2009,6,Cincinnati,68310.0,06/2009,69,044313,2.0
3,3,523-0006-0222-00,6110 Elder St,1436,510,1930,2009-06-24,120000,6,3,...,-84.390794,39.146337,2009,6,Cincinnati,56400.0,06/2009,94,044313,2.0
4,4,523-0006-0304-00,3811 Carlton Ave,1064,510,1918,2009-05-05,105000,5,3,...,-84.388793,39.145544,2009,5,Cincinnati,57530.0,05/2009,106,044313,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1265,1265,621-0024-0097-00,110 104 Carrington Ln,1412,550,1992,2023-06-05,250000,6,2,...,-84.301031,39.286614,2023,6,Loveland,12000.0,06/2023,32,044867,22.0
1266,1266,621-0024-0102-00,120 101 Carrington Ln,1095,550,1992,2023-06-02,177000,5,2,...,-84.301031,39.286614,2023,6,Loveland,12000.0,06/2023,32,044867,22.0
1267,1267,621-0024-0117-00,130 104 Carrington Ln,1412,550,1992,2023-04-07,200000,6,2,...,-84.301031,39.286614,2023,4,Loveland,12000.0,04/2023,32,044867,22.0
1268,1268,621-0024-0121-00,130 208 Carrington Ln,1412,550,1992,2023-04-28,210000,6,2,...,-84.301031,39.286614,2023,4,Loveland,12000.0,04/2023,32,044867,22.0
